# ALS applications

## Dzen dataset

Data comes from [dzen.ru](https://dzen.ru/) site and consists of likes which users put to text articles

### Columns
1. item_id - unique id of an item (article)
2. user_id - unique id of a user
3. source_id - unique id of an author. If two items have same source_id, then they come from one author
4. Name of item is name of the article
5. Raw dataset represents user_id and list of item_ids which user liked

In [1]:
#curl -O -J -L 'https://www.dropbox.com/s/ia4bvhuqg8kesee/zen_dataset.zip?dl=1'
#unzip zen_dataset.zip

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm.notebook import tqdm
import ast
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
item_names = pd.read_csv("zen_item_to_name.csv")
item_sources = pd.read_csv("zen_item_to_source.csv")
dataset = pd.read_csv("zen_ratings.csv", converters={'item_ids': ast.literal_eval})

In [4]:
item_names

,id,name
0,94962,Что обычно ожидало русских казачек в руках у к...
1,3972,Почему Россия решила строить новую скоростную ...
2,94644,"5 неприличных фактов об Андрее Макаревиче, кот..."
3,82518,"Что стало с красавицей Хмельницкой, которую му..."
4,53264,"Понять и Простить: Почему угонщики, бежавшие и..."
...,...,...
104498,36769,"Плюс один источник мифа о рыцарях, неспособных..."
104499,9190,Мой сад - малоуходный
104500,52731,Купил первую в жизни циркулярную пилу. Честный...
104501,72660,Решили предложить Марине помощь в лечении ч.10


In [5]:
item_sources

,id,source
0,94962,2919814402697966089
1,3972,3263022753228392991
2,94644,-3857390427602554682
3,82518,-9036908390349249792
4,53264,3353856219169766284
...,...,...
104498,36769,3818746211375738614
104499,9190,4975535765688979937
104500,52731,3720366796439288909
104501,72660,-7860042973720636310


In [6]:
dataset

,user_id,item_ids
0,993675863667353526,"[15267, 61075, 81203, 17066, 25471, 88427, 638..."
1,4250619547882954185,"[4555, 94644, 84972, 17774, 94962, 78217, 2485..."
2,3847785305345691076,"[1898, 26703, 16525, 86939, 55017, 31069, 4035..."
3,1785181112918558233,"[75601, 102458, 28716, 100694, 5757, 47104, 60..."
4,5078748097863903181,"[72260, 40825, 2615, 42549, 379, 100818, 56827..."
...,...,...
75905,4954138831959898373,"[11881, 55520, 63054, 48015, 66952, 103830, 21..."
75906,4967793435819938014,"[74697, 11830, 63858, 87245, 41956, 62089, 686..."
75907,7137764184903122777,"[10353, 1775, 103680, 29704, 9782, 13295, 9975..."
75908,2624987805086334956,"[24324, 18854, 73319, 66641, 64078, 97387, 426..."


In [7]:
total_interactions_count = dataset.item_ids.map(len).sum()
user_coo = np.zeros(total_interactions_count, dtype=np.int64)
item_coo = np.zeros(total_interactions_count, dtype=np.int64)
pos = 0

for user_id, item_ids in enumerate(tqdm(dataset.item_ids)):
    user_coo[pos : pos + len(item_ids)] = user_id
    item_coo[pos : pos + len(item_ids)] = item_ids
    pos += len(item_ids)
    
shape = (max(user_coo) + 1, max(item_coo) + 1)
user_item_matrix = sp.coo_matrix(
    (np.ones(len(user_coo)), (user_coo, item_coo)), shape=shape
)
user_item_matrix = user_item_matrix.tocsr()
sp.save_npz("data_train.npz", user_item_matrix)
# Cleanup memory. Later you need just data_train.npz
del user_coo
del item_coo
del dataset

  0%|          | 0/75910 [00:00<?, ?it/s]

In [8]:
# you could start here if you already done precomputing
user_item_matrix = sp.load_npz("data_train.npz")

In [9]:
user_item_matrix

<75910x104503 sparse matrix of type '<class 'numpy.float64'>'
	with 5792423 stored elements in Compressed Sparse Row format>

In [10]:
def sparce_matrix_report(matrix):
    print('Size of raw data:', matrix.data.nbytes / 10**6, 'Mb')
    print('Feedback matrix size:', matrix.shape)

In [11]:
sparce_matrix_report(user_item_matrix)

Size of raw data: 46.339384 Mb
Feedback matrix size: (75910, 104503)


In [12]:
item_weights = np.array(user_item_matrix.tocsc().sum(0))[0]
top_to_bottom_order = np.argsort(-item_weights)
item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
total_item_count = (item_weights > 0).sum()
total_user_count = user_item_matrix.shape[0]


def build_debug_dataset(user_item_matrix, item_pct: float, user_pct: float):
    '''Get given percent of top rated items and given percent of random users'''
    user_count = int(total_user_count * user_pct), 
    item_count = int(total_item_count * item_pct)
    item_ids = top_to_bottom_order[:item_count]
    user_ids = np.random.choice(
        np.arange(user_item_matrix.shape[0]), size=user_count, replace=False
    )
    train = user_item_matrix[user_ids]
    train = train[:, item_ids]
    return train

In [13]:
debug_dataset = build_debug_dataset(user_item_matrix, 0.03, 0.03)

sparce_matrix_report(debug_dataset)

Size of raw data: 0.52144 Mb
Feedback matrix size: (2277, 3011)


This is useful for debugging (just to save time).

**Final answers should use full dataset!!!**

## Split dataset matrix (5 points)

in the following way: for 20% of users (random) remove one like - this will be test data. The rest is train data. (10 points)

In [14]:
def split_data(ratings, test_ratio=0.2):
    ratings = ratings.copy()
    train_matrix = ratings.copy()
    test_matrix = sp.lil_matrix(ratings.shape)  # Создаем пустую LIL матрицу для тестовых данных

    non_zero_idx = ratings.nonzero()
    non_zero_pairs = list(zip(non_zero_idx[0], non_zero_idx[1]))  # Список пар (пользователь, элемент)

    np.random.seed(42)
    np.random.shuffle(non_zero_pairs)

    num_test = int(np.floor(test_ratio * len(non_zero_pairs)))
    for idx in range(num_test):
        user, item = non_zero_pairs[idx]
        test_matrix[user, item] = ratings[user, item]  # Копируем оценку в тестовую матрицу
        train_matrix[user, item] = 0  # Удаляем оценку из обучающей матрицы

    train_matrix.eliminate_zeros()
    test_matrix = test_matrix.tocsr()

    return train_matrix, test_matrix

In [15]:
# train_ratings, test_ratings = split_data(user_item_matrix)
train_ratings, test_ratings = split_data(debug_dataset)

## Implement ALS, IALS (10 points each)

Note that due to size of data you need to implement algorithms with _sparce matrices_!

In [16]:
def eig_values(Q):
    return np.linalg.eigh(Q)[0]

def eig_vectors(Q):
    return np.linalg.eigh(Q)[1]

In [17]:
def calc_oposite_vectors(Y, A):
    B = Y.T.dot(Y) + lam * np.eye(k)
    C = A.dot(Y)
    return np.linalg.inv(B).dot(C.T).T

In [18]:
from scipy.sparse import diags

def als(ratings, k=10, lam=1e-4, iterations=10):
    '''Alternating Least Squares algorithm

    Args:
        ratings: sparce matrix of ratings
        k: size of embeddings
        lam: regularization term
        
    Returns:
        two matrices: of user embeddings and of item embeddings
    '''

    n_users, n_items = ratings.shape
    user_embeddings, item_embeddings = np.random.randn(n_users, k), np.random.randn(n_items, k)

    for iteration in range(iterations):
        for user in range(n_users):
            rated_items = ratings[user,:].nonzero()[1]
            if rated_items.size == 0:
                continue

            Q = item_embeddings[rated_items, :]
            R = ratings[user, rated_items].toarray()
            A = Q.T @ Q + lam * diags([1]*k)
            b = Q.T @ R.T
            user_embeddings[user] = np.linalg.solve(A, b).ravel()

        for item in range(n_items):
            rated_by_users = ratings[:,item].nonzero()[0]
            if rated_by_users.size == 0:
                continue

            P = user_embeddings[rated_by_users, :]
            R = ratings[rated_by_users, item].T.toarray()
            A = P.T @ P + lam * diags([1]*k)
            b = P.T @ R.T
            item_embeddings[item] = np.linalg.solve(A, b).ravel()

    return user_embeddings, item_embeddings

In [19]:
def ials(ratings, k=10, lam=1e-4, iterations=10, alpha=1e4):
    '''Implicit Alternating Least Squares algorithm

    Args:
        ratings: sparce matrix of ratings
        k: size of embeddings
        lam: regularization term

    Returns:
        two matrices: of user embeddings and of item embeddings
    '''
    n_users, n_items = ratings.shape
    user_embeddings = np.random.randn(n_users, k)
    item_embeddings = np.random.randn(n_items, k)

    # Sparse matrix format and lambda I precomputed
    lambda_eye = lam * np.eye(k)
    C = ratings.copy()
    C.data = 1 + alpha * C.data
    C = C.tocsr()  # Ensure C is in an efficient format for row operations

    for iteration in range(iterations):
        # Update user embeddings
        for user in range(n_users):
            Cu = diags(C[user].toarray().flatten())  # Flatten avoids reshaping issues
            Pu = ratings[user].toarray().flatten()  # Keep dense where necessary
            YTCuY = item_embeddings.T @ Cu @ item_embeddings + lambda_eye
            YTCuPu = item_embeddings.T @ (Cu @ Pu[:, None])

            user_embeddings[user, :] = np.linalg.solve(YTCuY, YTCuPu).ravel()

        # Update item embeddings
        for item in range(n_items):
            Ci = diags(C[:, item].toarray().flatten())
            Pi = ratings[:, item].toarray().flatten()
            XTCiX = user_embeddings.T @ Ci @ user_embeddings + lambda_eye
            XTCiPi = user_embeddings.T @ (Ci @ Pi[:, None])

            item_embeddings[item, :] = np.linalg.solve(XTCiX, XTCiPi).ravel()

    return user_embeddings, item_embeddings

## Compute MRR@100 metric for test users (10 points)

For ALS and IALS algorithms.

**Don't forget to use full dataset!**

In [20]:
def mmr(predictions, test_ratings, k=100):
    mrr_sum = 0.0
    num_users = test_ratings.shape[0]

    for user in range(num_users):
        pred_indices = predictions[user].indices
        pred_data = predictions[user].data
        if len(pred_data) == 0:
            continue

        top_k_indices = np.argsort(-pred_data)[:k]
        relevant_items = test_ratings[user].indices
        rank = k + 1
        for idx in top_k_indices:
            if pred_indices[idx] in relevant_items:
                rank = np.where(top_k_indices == idx)[0][0] + 1
                break
        if rank <= k:
            mrr_sum += 1.0 / rank

    mrr_value = mrr_sum / num_users
    return mrr_value

In [21]:
def get_predictions_sparse_matrix(user_embeddings, item_embeddings):
    predictions_dense_matrix = np.dot(user_embeddings, item_embeddings.T)
    predictions_sparse_matrix = sp.csr_matrix(predictions_dense_matrix)
    return predictions_sparse_matrix

In [22]:
n_users, n_items = train_ratings.shape
user_embeddings, item_embeddings = np.random.randn(n_users, 10), np.random.randn(n_items, 10)
lam=1e-4

for iteration in range(1):
    for item in range(n_items):
        print(item_embeddings[item])
        rated_by_users = train_ratings[:,item].nonzero()[0]
        if rated_by_users.size == 0:
            continue
        P = user_embeddings[rated_by_users, :]
        print(P.shape)
        R = train_ratings[rated_by_users, item].T.toarray()
        print(R.shape)
        A = P.T @ P + lam * diags([1]*10)
        b = P.T @ R.T
        item_embeddings[item] = np.linalg.solve(A, b).ravel()
        print(item_embeddings[item])
        break

[-0.33946025 -0.32888397 -0.17348526 -0.2852941   0.26993229  2.11541124
 -0.36835365  0.08905266  0.23431348 -0.51266497]
(152, 10)
(1, 152)
[-0.09991021 -0.10038556  0.05627523  0.12723511 -0.22659534  0.19181004
 -0.21621773  0.05628099  0.15153019  0.29597628]


In [23]:
user_embeddings, item_embeddings = als(train_ratings)
als_predictions = get_predictions_sparse_matrix(user_embeddings, item_embeddings)
mrr_als = mmr(als_predictions, test_ratings)
print(mrr_als)

0.007396325092638307


In [24]:
user_embeddings, item_embeddings = ials(train_ratings)
ials_predictions = get_predictions_sparse_matrix(user_embeddings, item_embeddings)
mrr_ials = mmr(ials_predictions, test_ratings)
print(mrr_ials)

0.006402242336097055


## Adjust hyperparameters of ALS and IALS to maximize MRR (20 points)

Main hyperparameters are regularization and weights for implicit case.

In [25]:
def train_als_model(k, lam, iterations):
    user_embeddings, item_embeddings = als(train_ratings, k, lam, iterations)
    als_predictions = get_predictions_sparse_matrix(user_embeddings, item_embeddings)
    return mmr(als_predictions, test_ratings)


In [26]:
def train_ials_model(k, lam, iterations, alpha):
    user_embeddings, item_embeddings = ials(train_ratings, k, lam, iterations, alpha)
    ials_predictions = get_predictions_sparse_matrix(user_embeddings, item_embeddings)
    return mmr(ials_predictions, test_ratings)

In [27]:
# import optuna

# def objective(trial):
#     # Suggest parameters within a more reasonable range
#     k = trial.suggest_int('k', 20, 100)
#     lam = trial.suggest_loguniform('lam', 1e-5, 1e-2)
#     iterations = trial.suggest_int('iterations', 3, 5)

#     # Assuming train_als_model exists and returns MRR directly
#     mrr = train_als_model(k, lam, iterations)
#     return -mrr  # Optuna minimizes, so return negative MRR

# # Define the study with a pruner
# pruner = optuna.pruners.SuccessiveHalvingPruner()
# study = optuna.create_study(pruner=pruner)
# study.optimize(objective, n_trials=15)

# print("Best parameters for ALS: ", study.best_params)

# # After optimization, use visualization tools to analyze the study
# from optuna.visualization import plot_optimization_history, plot_param_importances

# plot_optimization_history(study)
# plot_param_importances(study)


In [28]:
# import optuna
# import matplotlib.pyplot as plt

# def objective(trial):
#     k = trial.suggest_int('k', 100, 200)
#     lam = trial.suggest_loguniform('lam', 1e-5, 1e-3)
#     iterations = trial.suggest_int('iterations', 3, 5)
#     alpha = trial.suggest_int('alpha', 10, 100)
#     mrr = train_ials_model(k, lam, iterations, alpha)
#     return -mrr  # Negative MRR for minimization

# # Define the study with a pruner
# pruner = optuna.pruners.SuccessiveHalvingPruner()
# study = optuna.create_study(pruner=pruner)
# study.optimize(objective, n_trials=15)

# print("Best parameters for IALS: ", study.best_params)

# # Plotting the optimization history
# optuna.visualization.matplotlib.plot_optimization_history(study)
# plt.show()

# # Plotting the importance of hyperparameters
# optuna.visualization.matplotlib.plot_param_importances(study)
# plt.show()

# # Plotting the contour of hyperparameters interactions
# optuna.visualization.matplotlib.plot_contour(study)
# plt.show()


Best parameters for ALS:  {'k': 99, 'lam': 6.467812954095308e-05, 'iterations': 5}

Best parameters for IALS:  {'k': 89, 'lam': 1.3098043433293455e-05, 'iterations': 3, 'alpha': 70}


## Get similarities from item2item CF and SLIM (10 points each)

Item2item can be taken from the first homework, SLIM was implemented in the class.

Alternatively you could use libraries, but in this case you will need to convert dataset to their format.

You need to compute only item similarities, not predictions for users.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

item_user_matrix = user_item_matrix.T

In [30]:
i2i_similarity = cosine_similarity(item_user_matrix, dense_output=False)

In [30]:
import scipy.sparse as sp
from sklearn.linear_model import ElasticNet
from scipy.sparse import lil_matrix

n_items = user_item_matrix.shape[1]
slim_weights = lil_matrix((n_items, n_items))  # Use LIL for easy updates

for item_id in tqdm(range(n_items)):
    # Directly use the sparse slice of the matrix
    target = user_item_matrix[:, item_id].toarray().ravel()  # Keep as is for ElasticNet compatibility
    if item_id > 0:
        left_items = user_item_matrix[:, :item_id]
    else:
        left_items = sp.csr_matrix((user_item_matrix.shape[0], 0))  # Empty matrix with appropriate dimensions
    
    if item_id < n_items - 1:
        right_items = user_item_matrix[:, item_id + 1:]
    else:
        right_items = sp.csr_matrix((user_item_matrix.shape[0], 0))  # Empty matrix with appropriate dimensions

    other_items = sp.hstack([left_items, right_items], format='csr')  # Maintain sparse format

    model = ElasticNet(alpha=1e-4, l1_ratio=0.15, positive=True, fit_intercept=False, copy_X=False)
    model.fit(other_items, target)

    # Update weights directly in sparse format
    if item_id > 0:
        slim_weights[item_id, :item_id] = model.coef_[:item_id]
    if item_id < n_items - 1:
        slim_weights[item_id, item_id+1:] = model.coef_[item_id:]

slim_similarity = slim_weights.tocsr()  # Convert to CSR for better performance in subsequent operations


  0%|          | 0/104503 [00:00<?, ?it/s]

/Users/vladimirs/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(


## Compare similarities from four algorithms (20 points)

* plot distributions
* compute metrics (which you think are relevant)
* look at several top similar lists

Make conclusion how these methods differ in computing similarities

In [ ]:
non_zero_values = slim_similarity.data
plt.figure(figsize=(10, 6))
plt.hist(non_zero_values, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.title('Распределение ненулевых значений схожести SLIM')
plt.xlabel('Значения схожести')
plt.ylabel('Количество')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
non_zero_values = i2i_similarity.data
plt.figure(figsize=(10, 6))
plt.hist(non_zero_values, bins=50, color='blue', edgecolor='black', alpha=0.7)
plt.title('Распределение ненулевых значений схожести I2I')
plt.xlabel('Значения схожести')
plt.ylabel('Количество')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()



In [ ]:
import os
import sys

from collections import defaultdict, Counter
from tqdm.notebook import tqdm_notebook

import numpy as np
import pandas as pd
import scipy.stats as sps
import scipy.sparse as scsp
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ndcg_score, dcg_score, roc_auc_score, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

from joblib import Parallel, delayed

import tqdm
import json

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

sns.set()

In [ ]:
item_counts = pd.read_csv('zen_dataset/item_counts.csv', index_col=0)
item_meta = pd.read_csv('zen_dataset/item_meta.gz', compression='gzip', index_col=0)
user_ratings = pd.read_csv('zen_dataset/user_ratings.gz', compression='gzip', index_col=0)

In [ ]:
item_counts['itemId'] = item_counts['itemId'].apply(str)

In [ ]:
user_encoder = LabelEncoder().fit(user_ratings['userId'])
item_encoder = LabelEncoder().fit(item_counts['itemId'])

In [ ]:
all_items = item_counts['itemId']
indices = item_encoder.transform(all_items)
item_to_id = dict(zip(all_items, indices))

In [ ]:
import numba

item_ratings_ind = [numba.typed.List() for _ in range(len(item_encoder.classes_))]
user_ids = user_encoder.transform(user_ratings['userId'])

for user_id, items_with_ratings in tqdm_notebook(zip(user_ids, user_ratings['trainRatings']),
                                                 total=len(user_ratings)):
    item_ids, item_ratings = zip(*json.loads(items_with_ratings.replace("'", '"')).items())
    item_ids = [item_to_id[item_id] for item_id in item_ids]
    for item_id, rating in zip(item_ids, item_ratings):
        item_ratings_ind[item_id].append((user_id, rating))

In [ ]:
item_ratings_ind_nb = numba.typed.List(item_ratings_ind)

In [ ]:
@numba.njit()
def fit_one_item(item_ratings, j, n_iter=20, l2_reg=1.0, l1_reg=2.0):
    """
    Оптимизирует один столбец матрицы W
    * item_ratings -- список списков взаимодействий айтема,
      item_ratings[j] -- список взаимодействий айтема j с пользователями
    * j -- номер айтема, для которого ищем веса
    * n_iter -- количество итераций оптимизации
    * l1_reg, l2_reg -- коэффициенты регуляризации L1 и L2

    Возвращает dict: item -> вес, все ненулевые веса
    """
    n_items = len(item_ratings)
    per_item_positives = []

    item_interactions = set()
    for user, rating in item_ratings[j]:
        item_interactions.add(user)

    for i in range(n_items):
        positives = set()
        for user, rating in item_ratings[i]:
            if rating == 1 and user in item_interactions:
                positives.add(user)
        per_item_positives.append(positives)

    w = np.zeros(n_items)
    non_zero_items = set()
    for _ in range(n_iter):
        for k in range(n_items):
            if k == j:
                continue

            score = len(per_item_positives[k] & per_item_positives[j]) - l1_reg
            for i in non_zero_items:
              if i == k:
                continue

              score -= w[i] * len(per_item_positives[i] & per_item_positives[k])

              if score < 0:
                break

            score /= len(per_item_positives[k]) + l2_reg
            score = max(0.0, score)

            w[k] = score
            if w[k] > 1e-5:
                non_zero_items.add(k)

    non_zero_elements = {}
    for i, value in enumerate(w):
        assert value >= 0.0
        if value > 0:
            non_zero_elements[i] = value

    return non_zero_elements

In [ ]:
def get_item_meta(item_id):
    item_id = int(item_encoder.inverse_transform([item_id])[0])
    return item_meta[item_meta['itemId'] == item_id].iloc[0].to_dict()


def visualize_top(item_ratings_ind_nb, j, top=10):
    weights = fit_one_item(item_ratings_ind_nb, j)
    sorted_items = sorted(weights.items(), key=lambda x: x[1], reverse=True)[:top]

    item_ids, weights = zip(*sorted_items)
    items = map(get_item_meta, item_ids)
    anchor_item = get_item_meta(j)

    with pd.option_context('display.max_colwidth', 100):
        display(pd.DataFrame({
            anchor_item['title']: [item['title'] for item in items],
            'score': weights
        }))

In [ ]:
visualize_top(item_ratings_ind_nb, 50)

Conclusion:

SLIM
Основной недостаток SLIM — это относительная вычислительная сложность, так как не параллелиться.
ALS
Преимущества Метод эффективен для распределенных данных и хорошо параллелиться, так как можно считать независимо по каждой компоненте.
ALS не эффективен при работе с неявными данными.
Правильный подбор параметров алгоритма критичен для достижения высокой производительности и занимает продолжительное время
iALS
Преимущества: Метод позволяет предсказать неявные взаимодействия пользователей и айтемов.
Недостатки: Подбор параметров сложнее из-за дополнительного параметра -- alpha.
I2I
Преимущества: Легок в реализации, хорош когда связь между объектами стабильна, легко масштабируется
Недостатки: Проблема холодного старта и нестабильных данных